In [7]:
#importing libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime
pd.set_option('display.max_columns', None)

In [8]:
#importing dataset
df=pd.read_excel('data_cleaning.xlsx', sheet_name='Usage', header=None)
df.head

<bound method NDFrame.head of                                               0
0         3 PM  Mon 24th-Mar-2014___0.384 kwh  
1                5AM  15-Aug-2014___1.201  kwh 
2          __8PM Thu 20-Mar-2014____1.523 kwh  
3               6PM  23rd-Apr-2014___0.424  kwh
4     _1AM  Friday 19th-Dec-2014___0.209  kwh  
...                                         ...
8755         _1AM  Fri 07th-Nov-2014_0.084 kwh 
8756                _6AM 20-May-2014__1.027 kwh
8757   __2 AM  Tuesday 8th-Apr-2014___0.052 kwh
8758            9 PM 27th-Jan-2014_1.428  kwh  
8759          _12 PM 25th-Aug-2014_0.528  kwh  

[8760 rows x 1 columns]>

In [9]:
#changing column name
df.columns=['data']

In [10]:
#checking duplicates 
df.duplicated().sum()

6

In [11]:
#deleting duplicates
df.drop_duplicates(inplace=True)

In [12]:
#finding right pattern to clean and split data
patterntime=re.compile(r'[\s*|_*]*(\d+\s?[AM|PM]+)')
patterndate=re.compile(r'(\d+\w*-\w{3}-\d{4})')
patternusage=re.compile(r'(\d\.\d+)')

In [13]:
#checking patterns
df['data'].str.findall(patterntime)

0        [3 PM]
1         [5AM]
2         [8PM]
3         [6PM]
4         [1AM]
         ...   
8755      [1AM]
8756      [6AM]
8757     [2 AM]
8758     [9 PM]
8759    [12 PM]
Name: data, Length: 8754, dtype: object

In [14]:
df['data'].str.findall(patterndate)


0       [24th-Mar-2014]
1         [15-Aug-2014]
2         [20-Mar-2014]
3       [23rd-Apr-2014]
4       [19th-Dec-2014]
             ...       
8755    [07th-Nov-2014]
8756      [20-May-2014]
8757     [8th-Apr-2014]
8758    [27th-Jan-2014]
8759    [25th-Aug-2014]
Name: data, Length: 8754, dtype: object

In [15]:
df['data'].str.findall(patternusage)

0       [0.384]
1       [1.201]
2       [1.523]
3       [0.424]
4       [0.209]
         ...   
8755    [0.084]
8756    [1.027]
8757    [0.052]
8758    [1.428]
8759    [0.528]
Name: data, Length: 8754, dtype: object

In [16]:
#extracting patterns
time=df['data'].str.extract(patterntime)
date=df['data'].str.extract(patterndate)
usage=df['data'].str.extract(patternusage)



In [17]:
#changing columns names
time.columns=['time']
date.columns=['date']
usage.columns=['usage']

In [18]:
#splitting time column into am|pm and hour columns
time[['am/pm']]=time['time'].str.extract('(AM|PM)', expand = True)
time[['hour']]=time['time'].str.extract('(\d+)', expand = True)


In [19]:
display(time.head(), date.head(), usage.head())

,time,am/pm,hour
0,3 PM,PM,3
1,5AM,AM,5
2,8PM,PM,8
3,6PM,PM,6
4,1AM,AM,1


,date
0,24th-Mar-2014
1,15-Aug-2014
2,20-Mar-2014
3,23rd-Apr-2014
4,19th-Dec-2014


,usage
0,0.384
1,1.201
2,1.523
3,0.424
4,0.209


In [20]:
#creating new clean dataset
df_elec = pd.concat([time, date, usage], axis=1)
df_elec

,time,am/pm,hour,date,usage
0,3 PM,PM,3,24th-Mar-2014,0.384
1,5AM,AM,5,15-Aug-2014,1.201
2,8PM,PM,8,20-Mar-2014,1.523
3,6PM,PM,6,23rd-Apr-2014,0.424
4,1AM,AM,1,19th-Dec-2014,0.209
...,...,...,...,...,...
8755,1AM,AM,1,07th-Nov-2014,0.084
8756,6AM,AM,6,20-May-2014,1.027
8757,2 AM,AM,2,8th-Apr-2014,0.052
8758,9 PM,PM,9,27th-Jan-2014,1.428


In [21]:
#cleaning time column
df_elec['time']=df_elec['time'].replace(' ', '', regex=True)
df_elec

,time,am/pm,hour,date,usage
0,3PM,PM,3,24th-Mar-2014,0.384
1,5AM,AM,5,15-Aug-2014,1.201
2,8PM,PM,8,20-Mar-2014,1.523
3,6PM,PM,6,23rd-Apr-2014,0.424
4,1AM,AM,1,19th-Dec-2014,0.209
...,...,...,...,...,...
8755,1AM,AM,1,07th-Nov-2014,0.084
8756,6AM,AM,6,20-May-2014,1.027
8757,2AM,AM,2,8th-Apr-2014,0.052
8758,9PM,PM,9,27th-Jan-2014,1.428


In [22]:
#cleaning date column
df_elec['date'] = df_elec['date'].replace(("st", 'nd', 'rd', 'th'), "", regex=True)
df_elec

,time,am/pm,hour,date,usage
0,3PM,PM,3,24-Mar-2014,0.384
1,5AM,AM,5,15-Aug-2014,1.201
2,8PM,PM,8,20-Mar-2014,1.523
3,6PM,PM,6,23-Apr-2014,0.424
4,1AM,AM,1,19-Dec-2014,0.209
...,...,...,...,...,...
8755,1AM,AM,1,07-Nov-2014,0.084
8756,6AM,AM,6,20-May-2014,1.027
8757,2AM,AM,2,8-Apr-2014,0.052
8758,9PM,PM,9,27-Jan-2014,1.428


In [23]:
#sorting data by date column
df_elec = df_elec.sort_values('date').reset_index(drop=True)
df_elec

,time,am/pm,hour,date,usage
0,8PM,PM,8,01-Apr-2014,0.557
1,2AM,AM,2,01-Apr-2014,0.052
2,10PM,PM,10,01-Apr-2014,0.135
3,5PM,PM,5,01-Apr-2014,0.355
4,9AM,AM,9,01-Apr-2014,0.078
...,...,...,...,...,...
8749,7AM,AM,7,9-Sep-2014,0.712
8750,10PM,PM,10,9-Sep-2014,0.312
8751,1AM,AM,1,9-Sep-2014,0.147
8752,5PM,PM,5,9-Sep-2014,0.774


In [24]:
#extracting am/pm from time column
time[['hour']]=time['time'].str.extract('(AM|PM)', expand = True)

In [25]:
#creating new columns
df_elec[['day', 'month', 'year']]=df_elec['date'].str.split('-', expand=True)
df_elec

,time,am/pm,hour,date,usage,day,month,year
0,8PM,PM,8,01-Apr-2014,0.557,01,Apr,2014
1,2AM,AM,2,01-Apr-2014,0.052,01,Apr,2014
2,10PM,PM,10,01-Apr-2014,0.135,01,Apr,2014
3,5PM,PM,5,01-Apr-2014,0.355,01,Apr,2014
4,9AM,AM,9,01-Apr-2014,0.078,01,Apr,2014
...,...,...,...,...,...,...,...,...
8749,7AM,AM,7,9-Sep-2014,0.712,9,Sep,2014
8750,10PM,PM,10,9-Sep-2014,0.312,9,Sep,2014
8751,1AM,AM,1,9-Sep-2014,0.147,9,Sep,2014
8752,5PM,PM,5,9-Sep-2014,0.774,9,Sep,2014


In [26]:
#changing data types
df_elec['date'] = df_elec['date'].astype('datetime64[ns]')
df_elec['usage'] = df_elec['usage'].astype('float64')
df_elec.head()

,time,am/pm,hour,date,usage,day,month,year
0,8PM,PM,8,2014-04-01,0.557,01,Apr,2014
1,2AM,AM,2,2014-04-01,0.052,01,Apr,2014
2,10PM,PM,10,2014-04-01,0.135,01,Apr,2014
3,5PM,PM,5,2014-04-01,0.355,01,Apr,2014
4,9AM,AM,9,2014-04-01,0.078,01,Apr,2014


In [27]:
#formating basic date time format
def convert_datetime(row):
    str_time = f"{row['year']}-{row['month']}-{row['day']}-{row['hour']}-{row['am/pm']}"
    return pd.to_datetime(datetime.strptime(str_time,"%Y-%b-%d-%I-%p"))

In [28]:
#creating new columns
df_elec['datetime']=df_elec.apply(func=convert_datetime, axis=1, result_type='expand' )
df_elec

,time,am/pm,hour,date,usage,day,month,year,datetime
0,8PM,PM,8,2014-04-01,0.557,01,Apr,2014,2014-04-01 20:00:00
1,2AM,AM,2,2014-04-01,0.052,01,Apr,2014,2014-04-01 02:00:00
2,10PM,PM,10,2014-04-01,0.135,01,Apr,2014,2014-04-01 22:00:00
3,5PM,PM,5,2014-04-01,0.355,01,Apr,2014,2014-04-01 17:00:00
4,9AM,AM,9,2014-04-01,0.078,01,Apr,2014,2014-04-01 09:00:00
...,...,...,...,...,...,...,...,...,...
8749,7AM,AM,7,2014-09-09,0.712,9,Sep,2014,2014-09-09 07:00:00
8750,10PM,PM,10,2014-09-09,0.312,9,Sep,2014,2014-09-09 22:00:00
8751,1AM,AM,1,2014-09-09,0.147,9,Sep,2014,2014-09-09 01:00:00
8752,5PM,PM,5,2014-09-09,0.774,9,Sep,2014,2014-09-09 17:00:00


In [29]:
#creating new 24 hour column
df_elec['hour|24']= df_elec['datetime'].dt.hour
df_elec.head()

,time,am/pm,hour,date,usage,day,month,year,datetime,hour|24
0,8PM,PM,8,2014-04-01,0.557,01,Apr,2014,2014-04-01 20:00:00,20
1,2AM,AM,2,2014-04-01,0.052,01,Apr,2014,2014-04-01 02:00:00,2
2,10PM,PM,10,2014-04-01,0.135,01,Apr,2014,2014-04-01 22:00:00,22
3,5PM,PM,5,2014-04-01,0.355,01,Apr,2014,2014-04-01 17:00:00,17
4,9AM,AM,9,2014-04-01,0.078,01,Apr,2014,2014-04-01 09:00:00,9


# Question 1
**What is your average hourly electricity 
usage?**  

a.0.641kWh

b. 0.782kWh

c. 0.884kWh

d. 0.937kWh

In [30]:
df_elec['usage'].mean()
#Correct answer is B) 0.782kWh

0.7816121772903816

#Question 2
**What is your average electricity usage
perhour in February?**

a.0.760kWh

b. 0.784kWh

c. 0.808kWh

d. 0.834kWh

In [31]:
df_elec[df_elec['month']=='Feb']['usage'].mean()
#Correct answer is D)0.834kWh

0.8337347242921014

#Question 3
**Which day of the week has the 
highestaverage usage?**

a.Sunday

b. Monday

c. Tuesday

d. Wednesday

In [32]:
#creating new column for day names
df_elec['day_of_week'] = df_elec['datetime'].dt.day_name()

In [33]:
df_elec.groupby('day_of_week').mean().sort_values('usage', ascending=False)
#Correct answer is A) Sunday

,usage,hour|24
day_of_week,,
Sunday,0.898601,11.490778
Saturday,0.890300,11.499599
Friday,0.753744,11.500000
Thursday,0.745752,11.508420
Wednesday,0.733943,11.500000
Monday,0.725437,11.504006
Tuesday,0.724439,11.493976


#Question 4
**What is the highest amount of 
electricityused in a continuous 4 hour
period?**

a. 17.237kWh

b. 17.327kWh

c. 17.422kWh

d. 17.487kWh

In [34]:
#by using time series function finding answer
a= df_elec.sort_values('datetime').reset_index(drop=True)
a['usage'].rolling(4).sum().max()
#Correct answer is A) 17.237kWh

17.237000000000002

#Question 5
**Based on your historic electricity usage,
what would your annual cost of 
electricitybe under the "Monthly Flex"
contract?**

a. $1350.73

b. $1420.06

c. $1450.26

d. $1493.77


In [35]:
#importing dataset
data=pd.read_excel('/content/data_cleaning.xlsx', sheet_name='Contracts',usecols = "A:C, E:G, I:K",  header = 0, keep_default_na=False, skiprows=10)
data

,No Flex,Unnamed: 1,Unnamed: 2,Monthly Flex,Unnamed: 5,Unnamed: 6,Hourly Flex,Unnamed: 9,Unnamed: 10
0,,,,,,,,,
1,Flat Rate,0.21,$/kWh,January,0.2,$/kWh,12AM until 1AM,0.1,$/kWh
2,,,,February,0.19,$/kWh,1AM until 2AM,0.1,$/kWh
3,,,,March,0.17,$/kWh,2AM until 3AM,0.1,$/kWh
4,,,,April,0.18,$/kWh,3AM until 4AM,0.1,$/kWh
5,,,,May,0.22,$/kWh,4AM until 5AM,0.15,$/kWh
6,,,,June,0.27,$/kWh,5AM until 6AM,0.2,$/kWh
7,,,,July,0.24,$/kWh,6AM until 7AM,0.24,$/kWh
8,,,,August,0.19,$/kWh,7AM until 8AM,0.24,$/kWh
9,,,,September,0.18,$/kWh,8AM until 9AM,0.26,$/kWh


In [36]:
#extracting monthlt flex dataset
monthly_flex=pd.DataFrame([data['Monthly Flex'], data['Unnamed: 5']]).transpose()
monthly_flex=monthly_flex[1:13]
monthly_flex['Monthly Flex']=monthly_flex['Monthly Flex'].str[0 : 3]
monthly_flex

,Monthly Flex,Unnamed: 5
1,Jan,0.2
2,Feb,0.19
3,Mar,0.17
4,Apr,0.18
5,May,0.22
6,Jun,0.27
7,Jul,0.24
8,Aug,0.19
9,Sep,0.18
10,Oct,0.15


In [37]:
#creating dataframe for next steps
a = pd.DataFrame(df_elec.groupby('month')['usage'].sum().reset_index())
a

,month,usage
0,Apr,151.242
1,Aug,633.390
2,Dec,568.739
3,Feb,559.436
4,Jan,1042.660
5,Jul,1254.055
6,Jun,899.904
7,Mar,421.132
8,May,416.813
9,Nov,232.420


In [38]:
#creating a dataframe to find out annual cost
df_annual=pd.concat([a.set_index('month'),monthly_flex.set_index('Monthly Flex')], axis=1, join='inner').reset_index()
df_annual

,index,usage,Unnamed: 5
0,Apr,151.242,0.18
1,Aug,633.390,0.19
2,Dec,568.739,0.19
3,Feb,559.436,0.19
4,Jan,1042.660,0.2
5,Jul,1254.055,0.24
6,Jun,899.904,0.27
7,Mar,421.132,0.17
8,May,416.813,0.22
9,Nov,232.420,0.14


In [39]:
annual_usage=sum(df_annual['usage']*df_annual['Unnamed: 5'])
annual_usage
#Correct answer is B)$1420.06

1420.0582800000002

# Question 6
**Based on your historic electricity 
usage, which of the three contracts 
would produce the lowest annual 
cost?**

a.The No Flex plan

b. The Monthly Flex plan

c. The Hourly Flex plan

d. Impossible to Determin

In [40]:
#extracting hourly flex dataset
hourly_flex=pd.DataFrame([data['Hourly Flex'], data['Unnamed: 9']]).transpose()
hourly_flex=hourly_flex[1:25]
hourly_flex

,Hourly Flex,Unnamed: 9
1,12AM until 1AM,0.1
2,1AM until 2AM,0.1
3,2AM until 3AM,0.1
4,3AM until 4AM,0.1
5,4AM until 5AM,0.15
6,5AM until 6AM,0.2
7,6AM until 7AM,0.24
8,7AM until 8AM,0.24
9,8AM until 9AM,0.26
10,9AM until 10AM,0.2


In [41]:
# extracting no flex dataset 
no_flex_data=pd.DataFrame([data['No Flex'], data['Unnamed: 1']]).transpose()
no_flex_data=no_flex_data[1:2]
no_flex_data

,No Flex,Unnamed: 1
1,Flat Rate,0.21


In [42]:
# cost under no flex is 1436.87
no_flex = df_elec['usage'].sum()*0.21
no_flex

1436.86893

In [43]:
#finding cost under hourly flex contract
hour_usage = df_elec.groupby(by = ['hour|24']).agg({'usage':'sum'}).reset_index(drop=True).values.ravel()

In [44]:
#finding cost under hourly flex
annual_cost_hourly = np.sum(hour_usage * (list(hourly_flex['Unnamed: 9'])))
annual_cost_hourly

1367.8482900000001

In [45]:
print('hourly flex: ' ,annual_cost_hourly)
print('no flex: ' ,no_flex)
print('monthly flex: ' ,annual_usage)


hourly flex:  1367.8482900000001
no flex:  1436.86893
monthly flex:  1420.0582800000002


In [46]:
#Correct answer is C) The Hourly Flex plan